# Segmenting and Clustering Neighborhoods in city of Toranto,Canada

Here we are following few steps 
1. installing the required libraries
2. webscraping a url to form a data set

1.Install or import required libraries

In [1]:
import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')



Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1


In [2]:
!pip install bs4
from bs4 import BeautifulSoup
import requests


  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=ccff709e4265e6e27e545e1c82b2a4bc970376c0332f2138372de9fd016b2575
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


To form a data frame for the postalcode of canada using the following url link of wikipedia

In [3]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
html_data=requests.get(url).text

In [6]:
soup=BeautifulSoup(html_data,'html5lib')

In [7]:
canada_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        canada_contents.append(cell)


Tranform the data into a pandas dataframe The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

In [8]:
canada_df=pd.DataFrame(canada_contents)

In [9]:
canada_contents[0]

{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}

In [10]:
for data in canada_contents:
    postalcode=data["PostalCode"]
    borough  =data['Borough']
    neighborhood = data['Neighborhood']

    canada_df= canada_df.append({'PostalCode':postalcode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood  },
                                           ignore_index=True)

In [11]:
canada_df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [14]:
canada_df.shape

(206, 3)